In [5]:
import pandas as pd
from ydata_profiling import ProfileReport
df = pd.read_csv("engin_test.csv")

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Recency,Frequency,Monetary,R,F,M
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,157,10,1142.05,4,2,3
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,157,10,1142.05,4,2,3
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,157,10,1142.05,4,2,3
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,157,10,1142.05,4,2,3
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,157,10,1142.05,4,2,3


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      797885 non-null  object 
 1   StockCode    797885 non-null  object 
 2   Description  797885 non-null  object 
 3   Quantity     797885 non-null  int64  
 4   InvoiceDate  797885 non-null  object 
 5   Price        797885 non-null  float64
 6   Customer ID  797885 non-null  float64
 7   Country      797885 non-null  object 
 8   Recency      797885 non-null  int64  
 9   Frequency    797885 non-null  int64  
 10  Monetary     797885 non-null  float64
 11  R            797885 non-null  int64  
 12  F            797885 non-null  int64  
 13  M            797885 non-null  int64  
dtypes: float64(3), int64(6), object(5)
memory usage: 85.2+ MB


### Laisser que UK

In [8]:
# Supprimer les lignes où la valeur de "Country" n'est pas "United Kingdom"
df = df[df['Country'] == 'United Kingdom']

# Afficher le DataFrame résultant

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Recency,Frequency,Monetary,R,F,M
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,157,10,1142.05,4,2,3
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,157,10,1142.05,4,2,3
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,157,10,1142.05,4,2,3
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,157,10,1142.05,4,2,3
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,157,10,1142.05,4,2,3


## Engin Featuring.

### Time Diff

In [9]:
import pandas as pd

# Charger le dataset


# Convertir la colonne "InvoiceDate" en type de données datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Trier le DataFrame par client et date
df_sorted = df.sort_values(by=['Customer ID', 'InvoiceDate'])

# Calculer la différence de jours entre les achats pour chaque client
df_sorted['TimeDiff'] = df_sorted.groupby('Customer ID')['InvoiceDate'].diff().dt.days

# Calculer la fréquence d'achat (nombre moyen de jours entre les achats) pour chaque client
df_frequency = df_sorted.groupby('Customer ID')['TimeDiff'].mean().reset_index()

# Afficher les premières lignes du résultat
print(df_frequency.head())


   Customer ID   TimeDiff
0      12346.0   8.543478
1      12608.0   0.000000
2      12745.0   4.142857
3      12746.0  10.631579
4      12747.0   2.816406


In [10]:
df_frequency.shape

(5410, 2)

### Total Amount

In [11]:
import pandas as pd

# Charger le dataset

# Calculer le montant total des achats par client
df['TotalAmount'] = df['Quantity'] * df['Price']

# Regrouper par client et calculer la somme totale des montants
df_total_amount = df.groupby('Customer ID')['TotalAmount'].sum().reset_index()

# Afficher les premières lignes du résultat
print(df_total_amount.head())


   Customer ID  TotalAmount
0      12346.0       -51.74
1      12608.0       415.79
2      12745.0       723.85
3      12746.0       230.85
4      12747.0      8786.53


In [12]:
df_total_amount.shape

(5410, 2)

### Moyen achat

In [13]:
import pandas as pd

# Charger le dataset


# Calculer le montant total des achats par transaction
df['TotalAmount'] = df['Quantity'] * df['Price']

# Regrouper par client et calculer le montant moyen des achats
df_avg_amount = df.groupby('Customer ID')['TotalAmount'].mean().reset_index()

# Renommer la colonne pour refléter l'information
df_avg_amount.rename(columns={'TotalAmount': 'AverageAmount'}, inplace=True)

# Afficher les premières lignes du résultat
print(df_avg_amount.head())


   Customer ID  AverageAmount
0      12346.0      -1.100851
1      12608.0      25.986875
2      12745.0      32.902273
3      12746.0      11.542500
4      12747.0      34.188833


In [14]:
df_avg_amount.shape

(5410, 2)

In [15]:
df['Description'].nunique()

5265

### Score RFM

In [19]:
import pandas as pd
import numpy as np

# Convertir la colonne "InvoiceDate" en type de données datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Calculer la date la plus récente dans le dataset
most_recent_date = df['InvoiceDate'].max()

# Calculer la récence (Recency) en jours
df['Recency'] = (most_recent_date - df.groupby('Customer ID')['InvoiceDate'].transform('max')).dt.days

# Calculer la fréquence (Frequency)
df['Frequency'] = df.groupby('Customer ID')['Invoice'].transform('nunique')

# Calculer le montant (Monetary)
df['Monetary'] = df.groupby('Customer ID')['Price'].transform('sum')

# Définir les seuils pour chaque mesure RFM
recency_thresholds = [0, 30, 90, 180, float('inf')]
frequency_thresholds = [0, 1, 2, 5, float('inf')]
monetary_thresholds = [0, 100, 500, 1000, float('inf')]

# Fonction d'interpolation linéaire
def interpolate_score(value, thresholds):
    for i in range(len(thresholds) - 1):
        if value <= thresholds[i+1]:
            return i + (value - thresholds[i]) / (thresholds[i+1] - thresholds[i])

# Assigner les scores RFM en utilisant l'interpolation linéaire
df['R'] = df['Recency'].apply(lambda x: interpolate_score(x, recency_thresholds))
df['F'] = df['Frequency'].apply(lambda x: interpolate_score(x, frequency_thresholds))
df['M'] = df['Monetary'].apply(lambda x: interpolate_score(x, monetary_thresholds))

print(df.head())



  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  Recency  Frequency  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom      157         10   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom      157         10   
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom      157         10   
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom      157         10   
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom      157         10   

   Monetary         R    F    M  TotalAmount  
0   1142.05  2.744444  3.0  3.0         83.4 

In [17]:
df['RFM'] = df['RFM'].astype(int)

In [20]:
# Effectuer un groupby sur Customer ID pour les colonnes R, F, M et RFM
df_grouped_RFM = df.groupby('Customer ID').agg({'R': 'mean', 'F': 'mean', 'M': 'mean',}).reset_index()

# Afficher le DataFrame résultant
print(df_grouped_RFM.head())


   Customer ID         R         F       M
0      12346.0  3.000000  3.000000  2.1350
1      12608.0  3.000000  1.000000  0.2604
2      12745.0  3.000000  2.000000  0.6798
3      12746.0  3.000000  2.333333  0.7395
4      12747.0  0.033333  3.000000  3.0000


In [21]:
df_grouped_RFM.shape

(5410, 4)

### Merge

In [22]:
import pandas as pd

# Fusionner les dataframes en utilisant la colonne "Customer ID" comme clé de fusion
merged_df = pd.merge(df_avg_amount, df_total_amount, on='Customer ID', how='inner')
merged_df = pd.merge(merged_df, df_frequency, on='Customer ID', how='inner')
merged_df = pd.merge(merged_df, df_grouped_RFM, on='Customer ID', how='inner')
# merged_df = pd.merge(merged_df, df_country_amount, on='Customer ID', how='inner')

# Afficher le dataframe fusionné
print(merged_df.head())



In [23]:
merged_df.head()

,Customer ID,AverageAmount,TotalAmount,TimeDiff,R,F,M
0,12346.0,-1.100851,-51.74,8.543478,3.000000,3.000000,2.1350
1,12608.0,25.986875,415.79,0.000000,3.000000,1.000000,0.2604
2,12745.0,32.902273,723.85,4.142857,3.000000,2.000000,0.6798
3,12746.0,11.542500,230.85,10.631579,3.000000,2.333333,0.7395
4,12747.0,34.188833,8786.53,2.816406,0.033333,3.000000,3.0000


In [24]:
merged_df.info()

In [25]:
#merged_df.to_csv('Feature_engin_treated.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5410 entries, 0 to 5409
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Customer ID    5410 non-null   float64
 1   AverageAmount  5410 non-null   float64
 2   TotalAmount    5410 non-null   float64
 3   TimeDiff       5272 non-null   float64
 4   R              5410 non-null   float64
 5   F              5410 non-null   float64
 6   M              5410 non-null   float64
dtypes: float64(7)
memory usage: 338.1 KB
